In [ ]:
import numpy as np
from collections import Counter, defaultdict
import concurrent.futures

# Define ranks and suits
ranks = np.arange(13)  # Map ranks to integers 0-12 (2 to Ace)
suits = np.arange(4)   # Map suits to integers 0-3 (hearts, diamonds, clubs, spades)

# Create a deck with two identical sets of 52 cards (104 cards total)
deck = np.array([(rank, suit) for rank in ranks for suit in suits] * 2, dtype=[('rank', 'i4'), ('suit', 'i4')])

def print_deck(deck):
    """Print the entire deck to show all cards in both identical sets."""
    for card in deck:
        rank = card['rank']
        suit = card['suit']
        suit_name = ["Hearts", "Diamonds", "Clubs", "Spades"][suit]
        rank_name = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"][rank]
        print(f"{rank_name} of {suit_name}", end=", ")
    print("\n")

def print_hand(hand):
    """Print the hand in a readable 'rank of suit' format."""
    hand_cards = []
    for card in hand:
        rank = card['rank']
        suit = card['suit']
        suit_name = ["Hearts", "Diamonds", "Clubs", "Spades"][suit]
        rank_name = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"][rank]
        hand_cards.append(f"{rank_name} of {suit_name}")
    print("Hand:", ", ".join(hand_cards))



def deal_hand(deck):
    """
    Shuffle the deck and select five unique cards.
    """
    shuffled_deck = np.random.permutation(deck)
    hand_indices = np.random.choice(len(shuffled_deck), 5, replace=False)
    hand = shuffled_deck[hand_indices]
    sorted_hand = np.sort(hand, order='rank')
    return shuffled_deck, sorted_hand

def is_royal_flush(hand):
    """
    Check if a hand is a Royal Flush (10, J, Q, K, A of same suit)
    With debugging output
    """
    # Get suit and rank info
    suits = set(card['suit'] for card in hand)
    sorted_ranks = sorted(card['rank'] for card in hand)

    # Check conditions
    return len(suits) == 1 and sorted_ranks == [8, 9, 10, 11, 12]

def run_simulation(num_trials, worker_id=None):
    """Run simulation with proper random seed"""
    if worker_id is not None:
        # Use a different base seed for each worker
        base_seed = 12345  # Any number would work
        np.random.seed(base_seed + worker_id * 1000)  # Ensure well-separated seeds

    royal_flush_count = 0
    local_examples = []

    for _ in range(num_trials):
        _, hand = deal_hand(deck)
        if is_royal_flush(hand):
            royal_flush_count += 1
            if len(local_examples) < 10:
                local_examples.append(hand.copy())

    return royal_flush_count, local_examples

def run_simulation_parallel(trials, num_workers=16):  # Increased from 4 to 16 workers
    """
    Run parallel simulation with more workers for better performance.
    """
    # Calculate chunk size and remaining hands
    chunk_size = trials // num_workers
    remaining = trials % num_workers
    total_royal_flush_count = 0
    all_examples = []


    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
        # Create futures with worker IDs
        futures = []
        for i in range(num_workers):
            # Last worker gets any remaining trials
            worker_trials = chunk_size + (remaining if i == num_workers-1 else 0)
            futures.append(executor.submit(run_simulation, worker_trials, worker_id=i))

        # Collect and track results
        worker_results = []
        for i, future in enumerate(concurrent.futures.as_completed(futures), 1):
            rf_count, examples = future.result()
            worker_results.append((rf_count, examples))
            print(f"Worker {i}/{num_workers} completed: Found {rf_count} Royal Flushes")

    # Combine results
    for rf_count, examples in worker_results:
        total_royal_flush_count += rf_count
        all_examples.extend(examples)

    print(f"Total Royal Flushes found: {total_royal_flush_count}")

    return total_royal_flush_count, all_examples



def main():
    print("\nRunning simulation with different trial counts...")
    trial_counts = [1_000, 10_000, 100_000, 1_000_000, 10_000_000, 100_000_000]

    for trials in trial_counts:
        print(f"\nRunning {trials:,} trials...")
        royal_flush_count, examples = run_simulation_parallel(trials)
        probability = royal_flush_count / trials

        print(f"\nResults for {trials:,} trials:")
        print(f"Found {royal_flush_count} Royal Flushes")
        print(f"Probability: {probability:.8f}")

        if examples:
            print("\nExample Royal Flushes found:")
            for i, hand in enumerate(examples, 1):
                print(f"\nRoyal Flush #{i}:")
                print_hand(hand)

        # After each large simulation, flush output and show separation
        if trials >= 1_000_000:
            print("\n" + "="*50 + "\n")

if __name__ == "__main__":
    main()


Running simulation with different trial counts...

Running 1,000 trials...
Worker 1/16 completed: Found 0 Royal Flushes
Worker 2/16 completed: Found 0 Royal Flushes
Worker 3/16 completed: Found 0 Royal Flushes
Worker 4/16 completed: Found 0 Royal Flushes
Worker 5/16 completed: Found 0 Royal Flushes
Worker 6/16 completed: Found 0 Royal Flushes
Worker 7/16 completed: Found 0 Royal Flushes
Worker 8/16 completed: Found 0 Royal Flushes
Worker 9/16 completed: Found 0 Royal Flushes
Worker 10/16 completed: Found 0 Royal Flushes
Worker 11/16 completed: Found 0 Royal Flushes
Worker 12/16 completed: Found 0 Royal Flushes
Worker 13/16 completed: Found 0 Royal Flushes
Worker 14/16 completed: Found 0 Royal Flushes
Worker 15/16 completed: Found 0 Royal Flushes
Worker 16/16 completed: Found 0 Royal Flushes
Total Royal Flushes found: 0

Results for 1,000 trials:
Found 0 Royal Flushes
Probability: 0.00000000

Running 10,000 trials...
Worker 1/16 completed: Found 0 Royal Flushes
Worker 2/16 completed: F